In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
excelFile = 'Cubing Data.xlsx'
solutions = pd.read_excel(excelFile)

In [2]:
date = 'Solution Date & Time'
solve = 'Time Elapsed (seconds)'
ao5 = 'Current Ao5'
ao12 = 'Current Ao12'
ao100 = 'Current Ao100'
del solutions['Scramble']

splits = [0,15,20,25,30,45,60]
splitNames = ['Sub-15', 'Sub-20', 'Sub-25', 'Sub-30', 'Sub-45', 'Sub-60']
# solutions['Single Splits'] = pd.cut(solutions['Time Elapsed (seconds)'], splits, right = False, labels=splitNames)
# The line above isn't required, but it can show visually how each time is categorized.

In [3]:
now = datetime.datetime.now()
currentYear = now.year
activeYear = 0
activeMonth = 0

# Determine what range you want to count
solutionsPerRange = []
rangeLabels = []

for index, row in solutions.iterrows():
    if row[0].year != activeYear and row[0].year != currentYear: #when entering a new year
        activeYear = row[0].year
        solutionsPerRange.append(1)
        rangeLabels.append(str(activeYear))
    elif row[0].year == activeYear and row[0].year != currentYear: #when in a year != present year
        solutionsPerRange[-1] += 1
    else: #when in the present year we want monthly breakdowns
        if row[0].month != activeMonth: #when entering a new month
            activeMonth = row[0].month
            solutionsPerRange.append(1)
            rangeLabels.append(datetime.date(currentYear, activeMonth, 1).strftime('%B %Y'))
        else: #when in any month
            solutionsPerRange[-1] += 1

# print(solutionsPerRange)
# print(rangeLabels)

ranges = [0]
nextEnd = 0
for x in range(len(solutionsPerRange)):
    nextEnd += solutionsPerRange[x]
    ranges.append(nextEnd)

# print(ranges)

# Stacked plots need to have all of one kind of data together (e.g., all the sub-15s in one list).
# Therefore, splitCounter needs to be rewritten to run through each range, count the sub-x times,
# then run again and count the next set of sub-x times.

# A nicer alternative may be to run through each date range and collect how many of each time there are,
# then transpose that matrix. Could be easier to implement and prevents us from having to run through each 
# date range multiple times to get more data.
# This alternative was chosen and is implemented in the cell below.

totalSplits = []

for x in range(len(ranges)-1):
    rangeStart = ranges[x] + 1
    rangeEnd = ranges[x+1] + 1
    currentRange = solutions.iloc[rangeStart:rangeEnd]   
    splitCount = [0, 0, 0, 0, 0, 0]
    for index, row in currentRange.iterrows():
        if row[1] < 15:
            splitCount[0] += 1
        elif row[1] < 20:
            splitCount[1] += 1
        elif row[1] < 25:
            splitCount[2] += 1
        elif row[1] < 30:
            splitCount[3] += 1
        elif row[1] < 45:
            splitCount[4] += 1
        else:
            splitCount[5] += 1
    totalSplits.append(splitCount)

# print(totalSplits)

[[0, 0, 2, 4, 62, 1], [7, 346, 1569, 1233, 753, 25], [41, 2217, 4444, 158, 2, 0], [4, 72, 146, 0, 0, 0], [30, 589, 530, 0, 0, 0], [24, 608, 373, 0, 0, 0], [58, 904, 378, 0, 0, 0], [6, 55, 13, 0, 0, 0], [17, 355, 174, 0, 0, 0], [24, 309, 117, 0, 0, 0], [60, 1003, 609, 0, 0, 0]]


In [13]:
# Prepare the data for plotting
plotData = pd.DataFrame(totalSplits)
plotData = plotData.transpose()
plotData.index = splitNames
plotData.columns = rangeLabels

# Normalize the data
normalizedData = plotData.divide(plotData.sum(axis=0), axis=1)
normalizedData = normalizedData * 100

# Plot the data
plt.stackplot(range(len(rangeLabels)), normalizedData, labels=splitNames)
plt.legend(loc='upper left')
plt.margins(0,0)
plt.title('Improvement over time')
plt.xticks(range(len(rangeLabels)),rangeLabels, rotation=45)
plt.show()

In [ ]:
#TODO:
#Progress evaluator for Ao5, 12, 100, etc.
#PB Tracker
#Weekday Evaluator
#Day-by-day PBs